In [88]:
!ls

architecture_diagram.png    llama-official-user-prompt-3.1-with-CoT
balanced_func_calling	    llama-user-prompt
balanced-json-modeagentic   llama-user-prompt-3.1
balanced_singleturn_merged  LOG_GUIDE.md
bfcl			    Notebooks
bfcl.egg-info		    openfunctions_evaluation.py
CHANGELOG.md		    Pre-Process-Downsampled-Nous.ipynb
CONTRIBUTING.md		    pyproject.toml
conversation_patterns.csv   qwen-re-run
CoT-ToolAce.ipynb	    README.md
data			    re-run-70B-FC
data_live.csv		    re-score
data_multi_turn.csv	    result
data_non_live.csv	    score
data_overall.csv	    state.json
dataset_info.json	    SUPPORTED_MODELS.md
Detailed-EDA-ToolACE.ipynb  test_case_ids_to_generate.json
Detailed-EDA-XLAM.ipynb     TEST_CATEGORIES.md
Down-sample-Nous.ipynb	    transformed_func_calling
ft-data			    transformed_toolace
glaive-balanced		    utils
hermes-function-calling-v1


In [212]:
from datasets import load_from_disk, DatasetDict
import json
import re

In [213]:
#These three need some work
# 1. human -> user
# 2. GPT -> Assistant
# 3. Change System prompt
# 4. Adjust and grab the JSON 
# 5. Remove XML tags from assistant and tool call
# 6. Fix the format in assistant tool calls
# 7. Fix the format in tool responses

In [ ]:
#Needs no XML tags removed but just conversion to python in System list-should be easier
json_agentic = load_from_disk("balanced-json-modeagentic")

In [ ]:
glaive_balanced	= load_from_disk("glaive-balanced")

In [290]:
func_calling = load_from_disk("balanced_func_calling")

In [318]:
single_turn = load_from_disk("balanced_singleturn_merged")

In [189]:
json_agentic['train'][45]

{'id': '8f52a9f4-9b46-4d04-8b39-94e0eb2b4c7e',
 'conversations': [{'from': 'system',
   'value': "You are a helpful assistant that answers in JSON. Here's the json schema you must adhere to:\n<schema>\n{'type': 'object', 'properties': {'translated_statement': {'type': 'string', 'description': 'The English translation of the given statement.', 'examples': ['Life is a beautiful gift, and we should enjoy it to the fullest.', '']}}, 'required': ['translated_statement']}\n</schema>\n"},
  {'from': 'human',
   'value': "I'm working on a translation project where I need to ensure that the translated English text is free from any profanity. The translations are for a wide audience, including children, so it's critical that the content is appropriate. Could you assist me by providing a JSON schema that validates the translated text to confirm there's no profanity? The text I need to validate is 'Life is a beautiful gift, and we should enjoy it to the fullest.'"},
  {'from': 'gpt',
   'value': '

In [190]:
def preprocess_conversation(example):
    # Convert roles
    for conv in example["conversations"]:
        if conv["from"] == "human":
            conv["from"] = "user"
        elif conv["from"] == "gpt":
            conv["from"] = "assistant"
    return example


# Load dataset
glaive_balanced = load_from_disk("balanced-json-modeagentic")

# Apply preprocessing
processed_dataset = glaive_balanced.map(preprocess_conversation)

# Save dataset
processed_dataset.save_to_disk("json-agentic-balanced-final")


Map:   0%|          | 0/303 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/303 [00:00<?, ? examples/s]

In [197]:
processed_dataset = load_from_disk("json-agentic-balanced-final")

In [199]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations', 'category', 'subcategory', 'schema'],
        num_rows: 303
    })
})

In [195]:
processed_dataset['train'][0]

{'id': '2d7a9111-0af2-4f0c-8804-4023d33246ee',
 'conversations': [{'from': 'system',
   'value': "You are a helpful assistant that answers in JSON. Here's the json schema you must adhere to:\n<schema>\n{'Conversational Dynamics': {'type': 'object', 'properties': {'current_context': {'type': 'string'}, 'previous_conversations': {'type': 'array', 'items': {'type': 'object', 'properties': {'speaker': {'type': 'string'}, 'utterance': {'type': 'string'}}, 'required': ['speaker', 'utterance']}}, 'retrieved_summary': {'type': 'string'}, 'planned_response': {'type': 'string'}, 'inner_thoughts': {'type': 'string'}, 'memo_on_conversation': {'type': 'string'}}, 'required': ['current_context', 'previous_conversations', 'retrieved_summary', 'planned_response', 'inner_thoughts', 'memo_on_conversation']}}\n</schema>\n"},
  {'from': 'user',
   'value': "I'm working on a project that involves a Simulacrum Agent, which is an AI designed to simulate human-like conversations. The agent has had several int

In [220]:
glaive_balanced

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations', 'tools', 'category', 'subcategory', 'task', 'source'],
        num_rows: 498
    })
})

In [219]:
glaive_balanced['train'][0]

{'id': '4226ad4f-4c17-4556-a946-2ec345577079',
 'conversations': [{'from': 'system',
   'value': 'You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions.Here are the available tools:<tools> [{\'type\': \'function\', \'function\': {\'name\': \'calculate_interest\', \'description\': \'Calculate the interest earned on an investment\', \'parameters\': {\'type\': \'object\', \'properties\': {\'principal\': {\'type\': \'number\', \'description\': \'The principal amount of the investment\'}, \'rate\': {\'type\': \'number\', \'description\': \'The interest rate per period\'}, \'time\': {\'type\': \'number\', \'description\': \'The time period in years\'}}, \'required\': [\'principal\', \'rate\', \'time\']}}}, {\'type\': \'function\', \'function\': {\'name\': \'calculate_age\', \'description\': \'Calculate the 

In [221]:
def update_conversations(conv_list):
   return [{**conv, 'from': 'user' if conv['from'] == 'human' else 'assistant' if conv['from'] == 'gpt' else conv['from']} for conv in conv_list]

def update_dataset(ds):
   ds['train'] = ds['train'].map(lambda x: {**x, 'conversations': update_conversations(x['conversations'])})
   return ds

In [223]:
glaive_balanced = update_dataset(glaive_balanced)

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

In [230]:
glaive_balanced['train'][0]

{'id': '4226ad4f-4c17-4556-a946-2ec345577079',
 'conversations': [{'from': 'system',
   'value': 'You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.You may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions.Here are the available tools:<tools> [{\'type\': \'function\', \'function\': {\'name\': \'calculate_interest\', \'description\': \'Calculate the interest earned on an investment\', \'parameters\': {\'type\': \'object\', \'properties\': {\'principal\': {\'type\': \'number\', \'description\': \'The principal amount of the investment\'}, \'rate\': {\'type\': \'number\', \'description\': \'The interest rate per period\'}, \'time\': {\'type\': \'number\', \'description\': \'The time period in years\'}}, \'required\': [\'principal\', \'rate\', \'time\']}}}, {\'type\': \'function\', \'function\': {\'name\': \'calculate_age\', \'description\': \'Calculate the 

In [231]:
def create_new_system_prompt(tools_json):
    base_prompt = """You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke.
"""
    return base_prompt + "\n" + tools_json

def update_conversations(conv_list, tools_json):
    # Update system prompt
    conv_list[0]['value'] = create_new_system_prompt(tools_json)
    return conv_list

def update_dataset(ds):
    ds['train'] = ds['train'].map(lambda x: {
        **x,
        'conversations': update_conversations(x['conversations'], x['tools']),
        'tools': None  # Drop the tools field
    })
    return ds

In [232]:
glaive_balanced = update_dataset(glaive_balanced)

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

In [234]:
glaive_balanced['train'][0]

{'id': '4226ad4f-4c17-4556-a946-2ec345577079',
 'conversations': [{'from': 'system',
   'value': 'You are an expert in composing functions. You are given a question and a set of possible functions.\nBased on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n\nIf you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\nYou SHOULD NOT include any other text in the response.\n\nHere is a list of functions in JSON format that you can invoke.\n\n[{"type": "function", "function": {"name": "calculate_interest", "description": "Calculate the interest earned on an investment", "parameters": {"type": "object", "properties": {"principal": {

In [235]:
def format_tool_call(json_str):
   # Remove XML tags and convert JSON-like string to function call format
   # From: "<tool_call>\n{'name': 'calculate_interest', 'arguments': {'principal': 5000, 'rate': 0.05, 'time': 3}}\n</tool_call>"
   # To: "[calculate_interest(principal=5000, rate=0.05, time=3)]"
   
   import ast
   # Remove XML tags and get the dict
   if "<tool_call>" in json_str:
       json_str = json_str.replace("<tool_call>", "").replace("</tool_call>", "").strip()
   
   # Parse string to dict
   data = ast.literal_eval(json_str)
   
   # Format to function call style
   args = [f"{k}={v}" for k,v in data['arguments'].items()]
   return f"[{data['name']}({', '.join(args)})]"

def format_tool_response(json_str):
   # Remove XML tags and convert to just the value
   # From: "<tool_response>\n{'interest': 750}\n</tool_response>"
   # To: "750"
   import ast
   if "<tool_response>" in json_str:
       json_str = json_str.replace("<tool_response>", "").replace("</tool_response>", "").strip()
   
   # Parse string to dict
   data = ast.literal_eval(json_str)
   
   # Return first value
   return str(list(data.values())[0])

def update_conversation_formats(conv_list):
   for conv in conv_list:
       if conv['from'] == 'assistant' and "<tool_call>" in conv['value']:
           conv['value'] = format_tool_call(conv['value'])
       elif conv['from'] == 'tool' and "<tool_response>" in conv['value']:
           conv['value'] = format_tool_response(conv['value'])
   return conv_list

def update_dataset(ds):
   ds['train'] = ds['train'].map(lambda x: {
       **x,
       'conversations': update_conversation_formats(x['conversations'])
   })
   return ds

In [236]:
glaive_balanced = update_dataset(glaive_balanced)

Map:   0%|          | 0/498 [00:00<?, ? examples/s]

In [240]:
glaive_balanced['train']

Dataset({
    features: ['id', 'conversations', 'tools', 'category', 'subcategory', 'task', 'source'],
    num_rows: 498
})

In [241]:
!pwd

/home/sanyambhutani/gorilla/berkeley-function-call-leaderboard


In [242]:
glaive_balanced.save_to_disk("/home/sanyambhutani/glaive-balanced-ready-to-train")

Saving the dataset (0/1 shards):   0%|          | 0/498 [00:00<?, ? examples/s]

In [244]:
!zip -r /home/sanyambhutani/glaive-final.zip /home/sanyambhutani/glaive-balanced-ready-to-train/

  adding: home/sanyambhutani/glaive-balanced-ready-to-train/ (stored 0%)
  adding: home/sanyambhutani/glaive-balanced-ready-to-train/dataset_dict.json (stored 0%)
  adding: home/sanyambhutani/glaive-balanced-ready-to-train/train/ (stored 0%)
 (deflated 77%)sanyambhutani/glaive-balanced-ready-to-train/train/data-00000-of-00001.arrow
  adding: home/sanyambhutani/glaive-balanced-ready-to-train/train/state.json (deflated 38%)
  adding: home/sanyambhutani/glaive-balanced-ready-to-train/train/dataset_info.json (deflated 75%)


In [260]:
func_calling

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations', 'category', 'subcategory', 'task'],
        num_rows: 819
    })
})

In [291]:
func_calling['train'][2]

{'id': 'b099e2c1-8459-4848-bd4f-0385e0d45d59',
 'conversations': [{'from': 'system',
   'value': "You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.\n<tools>\n[{'type': 'function', 'function': {'name': 'set_thermostat_schedule', 'description': 'Sets a temperature schedule for a smart thermostat.', 'parameters': {'type': 'object', 'properties': {'thermostat_id': {'type': 'string', 'description': 'The unique identifier of the smart thermostat.'}, 'schedule': {'type': 'array', 'description': 'A list of temperature settings with corresponding days and times.', 'items': {'type': 'object', 'properties': {'day': {'type': 'string', 'description': 'The day of the week for the scheduled temperature change.'}, 'temperature': {'type': 'number', 'description': 'The temperature to set at the specified time.'},

In [292]:
def transform_system_prompt(example):
    try:
        system_prompt = next(conv['value'] for conv in example['conversations'] if conv['from'] == 'system')
        tools_match = re.search(r'<tools>\n(.*?)\n</tools>', system_prompt, re.DOTALL)
        if not tools_match:
            return example
            
        tools_text = tools_match.group(1)
        func_match = re.search(r"'function': ({.*?})}]", tools_text, re.DOTALL)
        if not func_match:
            return example
            
        func_def = func_match.group(1)
        
        new_prompt = f"""You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke.

{func_def}"""

        example['conversations'][0]['value'] = new_prompt
        return example
    except Exception as e:
        print(f"Error processing example: {e}")
        return example

transformed_dataset = func_calling.map(transform_system_prompt)

In [293]:
transformed_dataset['train'][0]

{'id': '82088305-310b-45cb-ac76-ab273503b5cd',
 'conversations': [{'from': 'system',
   'value': "You are an expert in composing functions. You are given a question and a set of possible functions.\nBased on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n\nIf you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\nYou SHOULD NOT include any other text in the response.\n\nHere is a list of functions in JSON format that you can invoke.\n\n{'name': 'get_camera_live_feed', 'description': 'Retrieves the live feed from a specified security camera.', 'parameters': {'type': 'object', 'properties': {'camera_id': {'type': 'string', 'de

In [294]:
def convert_tool_format(example):
    try:
        for conv in example['conversations']:
            if conv['from'] == 'gpt':
                # Extract content between tool_call tags
                tool_match = re.search(r'<tool_call>\n(.*?)\n</tool_call>', conv['value'], re.DOTALL)
                if tool_match:
                    tool_dict = eval(tool_match.group(1))  # Convert string to dict
                    
                    # Convert to functional format
                    args = tool_dict['arguments']
                    func_name = tool_dict['name']
                    args_str = ', '.join(f"{k}={repr(v)}" for k, v in args.items())
                    new_format = f"[{func_name}({args_str})]"
                    
                    conv['value'] = new_format
                    
            elif conv['from'] == 'tool':
                # Remove tool_response tags and keep just the JSON
                tool_match = re.search(r'<tool_response>\n(.*?)\n</tool_response>', conv['value'], re.DOTALL)
                if tool_match:
                    conv['value'] = tool_match.group(1)
                    
        return example
    except Exception as e:
        print(f"Error processing example: {e}")
        return example

# Apply both transformations
transformed_dataset = func_calling.map(transform_system_prompt).map(convert_tool_format)

In [295]:
transformed_dataset['train'][24]

{'id': 'f2348c7b-702e-43b0-83f3-71ec4f71d553',
 'conversations': [{'from': 'system',
   'value': "You are an expert in composing functions. You are given a question and a set of possible functions.\nBased on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n\nIf you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\nYou SHOULD NOT include any other text in the response.\n\nHere is a list of functions in JSON format that you can invoke.\n\n{'name': 'book_appointment', 'description': 'Books an appointment for a patient with a specific dentist at a given date and time.', 'parameters': {'type': 'object', 'properties': {'patient_id'

In [296]:
def convert_tool_responses(example):
   try:
       for conv in example['conversations']:
           if conv['from'] == 'tool':
               tool_match = re.search(r'<tool_response>\n(.*?)\n</tool_response>', conv['value'], re.DOTALL)
               if tool_match:
                   tool_data = json.loads(tool_match.group(1))
                   # Keep only the content without name and wrapper
                   conv['value'] = json.dumps(tool_data.get('content', {}))
       return example
   except Exception as e:
       print(f"Error processing example: {e}")
       return example

transformed_dataset = transformed_dataset.map(convert_tool_responses)

In [297]:
transformed_dataset['train'][242]

{'id': '0eafc207-5e0c-4e6e-a560-6d65d53ae37b',
 'conversations': [{'from': 'system',
   'value': 'You are an expert in composing functions. You are given a question and a set of possible functions.\nBased on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n\nIf you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\nYou SHOULD NOT include any other text in the response.\n\nHere is a list of functions in JSON format that you can invoke.\n\n{\'name\': \'install_package\', \'description\': \'Installs a Python package using pip.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'package_name\': {\'type\': \'string\', \'de

In [298]:
def rename_roles(example):
   for conv in example['conversations']:
       if conv['from'] == 'human':
           conv['from'] = 'user'
       elif conv['from'] == 'gpt':
           conv['from'] = 'assistant'
   return example

transformed_dataset = transformed_dataset.map(rename_roles)

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

In [302]:
transformed_dataset['train'][0]

{'id': '82088305-310b-45cb-ac76-ab273503b5cd',
 'conversations': [{'from': 'system',
   'value': "You are an expert in composing functions. You are given a question and a set of possible functions.\nBased on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n\nIf you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\nYou SHOULD NOT include any other text in the response.\n\nHere is a list of functions in JSON format that you can invoke.\n\n{'name': 'get_camera_live_feed', 'description': 'Retrieves the live feed from a specified security camera.', 'parameters': {'type': 'object', 'properties': {'camera_id': {'type': 'string', 'de

In [303]:
transformed_dataset.save_to_disk("func-calling-multi-turn-final")

Saving the dataset (0/1 shards):   0%|          | 0/819 [00:00<?, ? examples/s]

In [304]:
!pwd

/home/sanyambhutani/gorilla/berkeley-function-call-leaderboard


In [305]:
!ls

architecture_diagram.png       json-agentic-balanced-final
balanced_func_calling	       llama-official-user-prompt-3.1-with-CoT
balanced-json-modeagentic      llama-user-prompt
balanced_singleturn_merged     llama-user-prompt-3.1
bfcl			       LOG_GUIDE.md
bfcl.egg-info		       Notebooks
CHANGELOG.md		       openfunctions_evaluation.py
CONTRIBUTING.md		       Pre-Process-Downsampled-Nous.ipynb
conversation_patterns.csv      pyproject.toml
CoT-ToolAce.ipynb	       qwen-re-run
data			       README.md
data_live.csv		       re-run-70B-FC
data_multi_turn.csv	       re-score
data_non_live.csv	       result
data_overall.csv	       score
dataset_info.json	       state.json
Detailed-EDA-ToolACE.ipynb     SUPPORTED_MODELS.md
Detailed-EDA-XLAM.ipynb        test_case_ids_to_generate.json
Down-sample-Nous.ipynb	       TEST_CATEGORIES.md
ft-data			       transformed_func_calling
func-calling-multi-turn-final  transformed_toolace
glaive-balanced		       updated_func_calling
glaive-final.zip	       ut

In [306]:
!zip -r /home/sanyambhutani/func-calling-multi-turn-final.zip /home/sanyambhutani/gorilla/berkeley-function-call-leaderboard/func-calling-multi-turn-final/

  adding: home/sanyambhutani/gorilla/berkeley-function-call-leaderboard/func-calling-multi-turn-final/ (stored 0%)
  adding: home/sanyambhutani/gorilla/berkeley-function-call-leaderboard/func-calling-multi-turn-final/dataset_dict.json (stored 0%)
  adding: home/sanyambhutani/gorilla/berkeley-function-call-leaderboard/func-calling-multi-turn-final/train/ (stored 0%)
 (deflated 77%)sanyambhutani/gorilla/berkeley-function-call-leaderboard/func-calling-multi-turn-final/train/data-00000-of-00001.arrow
  adding: home/sanyambhutani/gorilla/berkeley-function-call-leaderboard/func-calling-multi-turn-final/train/state.json (deflated 38%)
  adding: home/sanyambhutani/gorilla/berkeley-function-call-leaderboard/func-calling-multi-turn-final/train/dataset_info.json (deflated 72%)


In [308]:
single_turn

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations', 'category', 'subcategory', 'task', 'dataset_source'],
        num_rows: 260
    })
})

In [312]:
single_turn['train'][12]

{'id': '74a08af3-845d-46d9-93d4-c4993b9f824a',
 'conversations': [{'from': 'system',
   'value': 'You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions.\n<tools>\n[{\'type\': \'function\', \'function\': {\'name\': \'create_itinerary\', \'description\': \'Create a new itinerary with a title, description, and group size.\', \'parameters\': {\'type\': \'object\', \'properties\': {\'title\': {\'type\': \'string\', \'description\': \'The title of the itinerary.\'}, \'description\': {\'type\': \'string\', \'description\': \'A brief description of the itinerary.\'}, \'group_size\': {\'type\': \'integer\', \'description\': \'The number of people in the group.\'}}, \'required\': [\'title\', \'description\', \'group_size\']}}}, {\'type\': \'function\', \'function\': {\'name\': \'add_kayaking_location\', \'des

In [321]:
def transform_dataset(example):
   if example.get('dataset_source') != 'func_calling_singleturn':
       return example

   try:
       system_prompt = example['conversations'][0]['value']
       tools_match = re.search(r'<tools>\n(.*?)\n</tools>', system_prompt, re.DOTALL)
       if tools_match:
           tools_text = tools_match.group(1)
           try:
               # Parse functions array
               funcs = eval(tools_text)
               func_def = funcs[0]
           except:
               return example

           new_prompt = f"""You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.

If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.

Here is a list of functions in JSON format that you can invoke.

{func_def}"""
           example['conversations'][0]['value'] = new_prompt

       for conv in example['conversations']:
           if conv['from'] == 'human':
               conv['from'] = 'user'
           elif conv['from'] == 'gpt':
               conv['from'] = 'assistant'
               if '<tool_call>' in conv['value']:
                   tool_match = re.search(r'<tool_call>\n(.*?)\n</tool_call>', conv['value'], re.DOTALL)
                   if tool_match:
                       tool_dict = eval(tool_match.group(1))
                       args = tool_dict['arguments']
                       func_name = tool_dict['name']
                       args_str = ', '.join(f"{k}={repr(v)}" for k, v in args.items())
                       conv['value'] = f"[{func_name}({args_str})]"

   except Exception as e:
       print(f"Error processing example: {e}")
       
   return example

In [322]:
transformed_dataset = single_turn.map(transform_dataset)

Map:   0%|          | 0/260 [00:00<?, ? examples/s]

In [323]:
transformed_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'conversations', 'category', 'subcategory', 'task', 'dataset_source'],
        num_rows: 260
    })
})

In [328]:
transformed_dataset['train'][2]

{'id': '6de2be31-985e-413a-ae33-4c1140070920',
 'conversations': [{'from': 'system',
   'value': "You are an expert in composing functions. You are given a question and a set of possible functions.\nBased on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n\nIf you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\nYou SHOULD NOT include any other text in the response.\n\nHere is a list of functions in JSON format that you can invoke.\n\n{'type': 'function', 'function': {'name': 'get_language_syntax', 'description': 'Retrieves the syntax and structural information of specified quantum programming languages.', 'parameters': {'t

In [329]:
def rename_gpt(example):
   for conv in example['conversations']:
       if conv['from'] == 'gpt':
           conv['from'] = 'assistant'
   return example

transformed_dataset = transformed_dataset.map(rename_gpt)

Map:   0%|          | 0/260 [00:00<?, ? examples/s]

In [331]:
transformed_dataset['train'][0]

{'id': '610f6af1-e515-4167-8c16-930b22025e9a',
 'conversations': [{'from': 'system',
   'value': "You are an expert in composing functions. You are given a question and a set of possible functions.\nBased on the question, you will need to make one or more function/tool calls to achieve the purpose.\nIf none of the functions can be used, point it out. If the given question lacks the parameters required by the function, also point it out. You should only return the function call in tools call sections.\n\nIf you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]\nYou SHOULD NOT include any other text in the response.\n\nHere is a list of functions in JSON format that you can invoke.\n\n{'type': 'function', 'function': {'name': 'set_ambient_mode', 'description': 'Configures the smart devices to create a specific ambiance based on a voice command.', 'parameters': {'type': 'obj

In [332]:
transformed_dataset.save_to_disk("/home/sanyambhutani/single-turn-func-call-and-agent-final")

Saving the dataset (0/1 shards):   0%|          | 0/260 [00:00<?, ? examples/s]

In [334]:
!zip -r "/home/sanyambhutani/final-singleturnfunccall.zip" "/home/sanyambhutani/single-turn-func-call-and-agent-final/"

  adding: home/sanyambhutani/single-turn-func-call-and-agent-final/ (stored 0%)
  adding: home/sanyambhutani/single-turn-func-call-and-agent-final/dataset_dict.json (stored 0%)
  adding: home/sanyambhutani/single-turn-func-call-and-agent-final/train/ (stored 0%)
  adding: home/sanyambhutani/single-turn-func-call-and-agent-final/train/data-00000-of-00001.arrow (deflated 75%)
  adding: home/sanyambhutani/single-turn-func-call-and-agent-final/train/state.json (deflated 39%)
  adding: home/sanyambhutani/single-turn-func-call-and-agent-final/train/dataset_info.json (deflated 74%)
